# Big Data - Statistics
## Phase one - Data extraction and modification

In [12]:
import numpy as np
import pandas as pd

import xml.etree.ElementTree as et

xtree = et.parse("Data/export.xml")
xroot = xtree.getroot()

df_cols = ["distance", "date"]
df_out = pd.DataFrame(columns = df_cols)

for node in xroot:
    if (node.attrib.get("type") == "HKQuantityTypeIdentifierDistanceCycling") :
        d_distance = node.attrib.get("value")
        d_date = node.attrib.get("startDate")[0:10]
        df_out = df_out.append(pd.Series([d_distance, d_date], index = df_cols), ignore_index=True)
        
print(df_out.to_string())

    distance        date
0        3.7  2019-03-07
1        3.7  2019-03-07
2        3.7  2019-03-08
3        3.7  2019-03-08
4        3.7  2019-03-09
5        3.7  2019-03-11
6        3.7  2019-03-12
7        3.7  2019-03-13
8        3.7  2019-03-14
9        3.7  2019-03-12
10       3.7  2019-03-13
11       3.7  2019-03-14
12        12  2019-04-23
13        12  2019-04-23
14        12  2019-04-20
15         4  2019-03-15
16       8.2  2019-03-16
17       8.2  2019-03-19
18       8.2  2019-03-20
19       8.2  2019-03-21
20       8.2  2019-03-22
21       8.2  2019-03-23
22       8.2  2019-03-26
23       8.2  2019-03-27
24       8.2  2019-03-28
25       8.2  2019-03-29
26       8.2  2019-03-30
27       8.2  2019-04-02
28       8.2  2019-04-03
29       4.1  2019-04-23
30       6.5  2019-04-24
31       8.7  2019-04-24
32       8.2  2019-04-16
33       8.2  2019-04-17
34       8.2  2019-04-18
35       8.2  2019-04-19
36       8.2  2019-04-04
37       8.2  2019-04-05
38       8.2  2019-04-06
